In [ ]:
# Spell_Series???

In [ ]:
import numpy as np

import DFS_Functions
import DFS_World_States
from DFS_Functions import adjacent_locations, chebyshev_distance, bresenham_line, calculate_full_path, check_opportunity_attacks, is_line_of_sight_clear, check_visibility, generate_pseudo_history
from DFS_World_States import world, world_grid_states

import DFS_Action_Series

from DFS_Universal_Rules import action_subactions, move_subactions, action_subactions, attack_subactions, free_subactions, subactions_req_targets, object_subactions, target_distance_scores, subactions_req_objects



In [ ]:
# the idea is that if the cast subaction (14) is in the action_series, this function will be called...
# this function will create a number of duplicates of all act_loc_obj_series with the cast subaction,
# and the cast series will be the various spells that the character could cast

In [ ]:
# the cast series may require it's own:
# - location series
# - object series
# - entity series
# ... and that could be a lot...

# so the representation of a spell being cast would look like
        # [[action_series],[location_series],[object_series],[entity_series],[spell_series], etc...] from the outside

# and 
        # [for each spell cast in an action_series: 
        #                       [specific_spell_subclass,
        #                       [spell_options_if_any],         # hardcoded such as shape water or open ended such as suggestion
        #                       [list_of_locations_it_can_target], 
        #                       [list_of_objects_it_can_target], 
        #                       [list_of_entities_it_can_target], 
        #                       ...
        # ] from the inside

class RuleBasedSpellSeriesDFS:
        def __init__(self, acting_entity, act_loc_obj_rew_series, spell_rules):
                self.acting_entity = acting_entity
                self.act_loc_obj_rew_series = act_loc_obj_rew_series
                self.spell_rules = spell_rules

        def check_rules(self, sequence, spell_rules, next_spell, acting_entity):
                return all(rule(sequence, next_spell, acting_entity) for rule in spell_rules)
                
        def dfs(self, action_series, location_series, object_series, current_sequence, acting_entity):
                if len(current_sequence) == len([x for x in action_series if x == 14]):
                        return current_sequence
                

                for spell in acting_entity.spells:
                        if self.check_rules(current_sequence + [spell], self.spell_rules, acting_entity):
                                return self.dfs(action_series, location_series, object_series, current_sequence + [spell], acting_entity)                        

                
                                                

        def generate_spell_series(self):
                spell_series = []
                for act_loc_obj_rew in self.act_loc_obj_rew_series:
                        action_series = act_loc_obj_rew[0]
                        location_series = act_loc_obj_rew[1]
                        object_series = act_loc_obj_rew[2]
                        reward = act_loc_obj_rew[3]

                        quality_threshold = np.percentile(reward, 85)

                        if reward >= quality_threshold:
                                for action in action_series:
                                        if action == 14:
                                                spell_series.append(self.dfs(action_series, location_series, object_series, [], self.acting_entity))
                return spell_series
        


In [ ]:
# Spell Series Rules:
# - spell slots
# - casting focus
# - hand requirements
# - sight required
# - spells per turn
# - 

def rule_spell_slot_requirement(sequence, next_spell, acting_entity):
    pseudo_spell_slots = acting_entity.spell_slots.copy()

    for spell in sequence:
        spell_slot_level_required = spell.level
        pseudo_spell_slots[spell_slot_level_required] -= 1

    spell_slot_level_required = next_spell.level
    if pseudo_spell_slots[spell_slot_level_required] == 0:
        return False

    return True


# How do main_hand, off_hand, and spellcasting focus interact with somatic and material components in spellcasting?

# Somatic Component requires 1 Free Hand
    # "If a spell requires a somatic component, the caster must have free use of at least one hand"

# Material Component requires 1 Free Hand
    # "A spellcaster must have a hand free to access a spell's material components—or to hold a spellcasting focus—but it can be the same hand that he or she uses to perform somatic components."

# Spellcasting Focus can replace Material Components
    # "A character can use... a spellcasting focus... in place of the [material] components specified for a spell."

# Sage Advice Compendium
# " Another example: a cleric’s holy symbol is emblazoned on her shield. 
#   She likes to wade into melee combat with a mace in one hand and a shield in the other. 
#   She uses the holy symbol as her spellcasting focus, 
#   so she needs to have the shield in hand when she casts a cleric spell that has a material component. 
#   If the spell, such as aid, also has a somatic component, 
#   she can perform that component with the shield hand and keep holding the mace in the other.
#   
#   If the same cleric casts cure wounds, she needs to put the mace or the shield away, 
#   because that spell doesn’t have a material component but does have a somatic component. 
#   She’s going to need a free hand to make the spell’s gestures. 
#   If she had the War Caster feat, she could ignore this restriction. "

# Aid [v, s, m]
# the material component gets replaced by the holy symbol (spellcasting focus) 
# because the somatic component can be performed with the hand that is holding the spellcasting focus, 
# the spell requires at minimum one hand to cast (both hold the spellcasting focus and perform the somatic component)

# Cure Wounds [v, s]
# because there is no material component, and the holy symbol/shield cannot replace a somatic component,
# the cleric needs to free up a hand (either the shield or the mace) to cast the spell
# the spell requires at minimum one hand, which must be capable of performing the somatic component (i.e. in most cases empty)

# Revivify [v, s, m*]
# because the material component cannot be replaced by a spellcasting focus (due to it being consumed by the spell),
# the cleric needs to drop the shield and hold the material component (diamond) in order to cast the spell
# the spell requires at minimum one hand (both hold the material component and perform the somatic component)


def rule_somatic_components_require_hands(sequence, next_spell, acting_entity):
    return True
        

In [ ]:


spell_rules = [
        rule_spell_slot_requirement,
        
]